In [ ]:
%load_ext autoreload
%autoreload 2
import plotly.io as pio
pio.renderers.default = "notebook"

In [ ]:
import os
path_parent = os.path.dirname(os.getcwd())
data_directory = os.path.join(path_parent, "data")
os.chdir(path_parent)

import plotly.graph_objects as go
import chart_studio.plotly as py
from plotly.offline import iplot
import plotly.figure_factory as ff
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
import pycrash
from pycrash.data.defaults.config import default_dict
mu_max = default_dict['mu_max']             # maximum available friction

In [ ]:
project_inputs = {'name':'Practice', 'pdesc':'single motion', 'sim_type':'SV', 'impact_type':'none',
                  'note':'single vehicle motion demo'}
proj = pycrash.Project(project_inputs)

In [ ]:
# for vehicle motion simluation, driver input is provided as a dataframe
end_time = 5  # 5 second simulation
t = list(np.arange(0, end_time+0.1, 0.1))  # create time array from 0 to end time from user
throttle = [0] * len(t)                                # no throttle
brake = [0] * len(t)                                   # no braking
steer = [0] * len(t)                                   # no steering
driver_input_dict = {'t':t, 'throttle':throttle, 'brake':brake, 'steer':steer}
driver_input_df = pd.DataFrame.from_dict(driver_input_dict)

vehicle_input_dict = {"year":2016,
"make":"Subaru",
"model":"WRX Sti",
"weight":3000,
"vin":"123abc",
"brake":0,
"steer_ratio":16.5,
"init_x_pos":0,
"init_y_pos":0,
"head_angle":0,
"width":6.6,
"length":20.66,
"hcg":2,
"lcgf":5.6,
"lcgr":7.76,
"wb":13.36,
"track":5.7,
"f_hang":3.2,
"r_hang":3.873,
"tire_d":2.716666667,
"tire_w":0.866666667,
"izz":3711,
"fwd":0,
"rwd":1,
"awd":0,
"A":100,
"B":41,
"k":1000,
"L":0,
"c":0,
"vx_initial":5,
"vy_initial":0,
"omega_z":0,
"driver_input":driver_input_df}

veh1 = pycrash.Vehicle('Veh1', vehicle_input_dict)

In [ ]:
# %% Change driver inputs
time = [0, 2.5, 10]
throttle = [0, 0, 0]
brake = [0, 0, 0]
steer = [0, 360, 360]
veh1.vx_initial = 20
veh1.vy_initial = 0
pio.renderers.default = "notebook"
veh1.time_inputs(time, throttle, brake, steer)

In [ ]:
# %% vehicle motion
motion = pycrash.SingleMotion('motion1', veh1)

In [ ]:
pio.renderers.default = "browser"
motion.plot_model()

In [ ]:
# %%
i = 24
i += 1
pio.renderers.default = "browser"
#motion.global_motion(len(motion.veh.model.t) - 1)
motion.global_motion(i)
print(f'Time = {motion.veh.model.t[i]}')

In [ ]:
pio.renderers.default = "browser"
motion.global_motion(len(motion.veh.model.t) - 1)
print(f'Time = {motion.veh.model.t[len(motion.veh.model.t) - 1]}')

### Tire Details

In [ ]:
pio.renderers.default = "notebook"
motion.tire_detail()

In [ ]:
import math
df_plot = motion.veh.model.copy()
df_plot['lf_latf_one'] = [ -1 * mu_max * math.sin(row) for row in df_plot.lf_alpha]
df_plot['lf_latf'] = df_plot['lf_latf_one'] * df_plot.lf_fz

df_plot['lf_longf_one'] = [-1 * mu_max * row for row in veh1.driver_input.brake]
df_plot['lf_app'] = df_plot['lf_longf_one'] * df_plot.lf_fz

df_plot['lf_max_fz'] = [mu_max * row for row in df_plot.lf_fz]
                        
figure_size = (800,450)

wheel_colors = ['rgb(0, 0, 255)', 'rgb(0, 255, 0)', 'rgb(153, 0, 204)', 'rgb(255, 102, 0)']

fig = go.Figure()

fig.add_trace(go.Scatter(x = df_plot.t, y = df_plot.lf_latf,
                    mode = 'lines',
                    name = 'lateral applied',
                    line = dict(color = wheel_colors[0], width = 1, dash = 'solid')))
fig.add_trace(go.Scatter(x = df_plot.t, y = df_plot.lf_app,
                    mode = 'lines',
                    name = 'long applied',
                    line = dict(color = 'rgb(0, 0, 0)', width = 1, dash = 'solid')))
fig.add_trace(go.Scatter(x = motion.veh.model.t, y = motion.veh.model.lf_alpha * 180 / math.pi,
                    mode = 'lines',
                    name = 'alpha',
                    line = dict(color = wheel_colors[1], width = 1, dash = 'solid')))
fig.add_trace(go.Scatter(x = motion.veh.model.t, y = df_plot.lf_max_fz,
                    mode = 'lines',
                    name = 'lf max Fz',
                    line = dict(color = 'rgb(255, 0, 0)', width = 1,
                    dash = 'solid')))
fig.add_trace(go.Scatter(x = motion.veh.model.t, y = df_plot.lf_lock * 100,
                    mode = 'lines',
                    name = 'lf lock',
                    line = dict(color = 'rgb(255, 0, 0)', width = 1,
                    dash = 'dash')))                       
fig.show()

In [ ]:
math.sin(164 * math.pi / 180)

In [ ]:
math.cos(28 * math.pi / 180)

In [ ]:
math.sin(-90 * math.pi / 180)

In [ ]:
-1 * np.arctan2(1,0) * 180 / math.pi

In [ ]:
math.sin(-90 * math.pi / 180)

In [ ]:
math.cos(-45 * math.pi / 180)

In [ ]:
# %%
motion.veh.model.head(150)

In [ ]:
ax = motion.veh.model.plot(x = 't', y='Dy', figsize = (16,9))
motion.veh.model.plot(x = 't', y='vy', secondary_y=True, ax=ax)

In [ ]:
ax = motion.veh.model.plot(x = 't', y='lf_alpha', figsize = (16,9))
motion.veh.model.plot(x = 't', y= 'lr_alpha', secondary_y=False, ax=ax)

In [ ]:
stopped = motion.veh.model[motion.veh.model.t > 1].copy()
stopped.vy.mean()

In [ ]:
motion.CG_motion()

### atan2

In [ ]:
x = np.array([-1, +1, +1, -1])
y = np.array([-1, -1, +1, +1])
np.arctan2(y, x) * 180 / np.pi

In [ ]:
os.getcwd()

In [ ]:
motion.veh.model.to_csv('modeldf.csv')